In [4]:
import numpy as np

n_samples = 50000
t = np.linspace(0, 2*np.pi, n_samples)

# θ1 = 180 * np.sin(t + 0)                         # No shift
# θ2 = 120 * np.sin(2*t + np.pi/6)                 # Phase shifted
# θ3 = 127.5 * np.sin(1.5*t + np.pi/3) - 52.5      # Shift + offset
# θ4 = 400 * np.sin(2.5*t + np.pi/2)
# θ5 = 122.5 * np.sin(t + np.pi/4) - 2.5
# θ6 = 400 * np.sin(3*t + np.pi/5)

θ1 = 170 * np.sin(t)                                     # [-170, 170]
θ2 = 80 * np.sin(2*t + np.pi/6)                          # [-75, 85]
θ3 = 120 * np.sin(1.5*t + np.pi/3) - 52.5                         # [-170, 70]
θ4 = 200 * np.sin(2.5*t + np.pi/2)                               # [-200, 200]
θ5 = 120 * np.sin(t + np.pi / 4) - 2.5                         # [-120, 120]
θ6 = 400 * np.sin(3*t + np.pi/5)                                 # [-400, 400]


In [7]:
# gpt DF parameters

def dh_matrix(theta, d, a, alpha):
    ct, st = np.cos(theta), np.sin(theta)
    ca, sa = np.cos(alpha), np.sin(alpha)
    return np.array([
        [ct, -st*ca, st*sa, a*ct],
        [st, ct*ca, -ct*sa, a*st],
        [0, sa, ca, d],
        [0, 0, 0, 1]
    ])

# forward kinemarics
def fk_irb4600(joint_angles_deg):
    theta = np.radians(joint_angles_deg)

    # DH parameters
    dh_params = [
        [theta[0], 0, 160, np.pi/2],
        [theta[1], 0, 590, 0],
        [theta[2], 0, 200, np.pi/2],
        [theta[3], 723, 0, np.pi/2],
        [theta[4], 0, 0, -np.pi/2],
        [theta[5], 200, 0, 0]
    ]

    T = np.eye(4)
    for p in dh_params:
        T = T @ dh_matrix(*p)
    # print(T)
    position = T[:3, 3]
    rotation = T[:3, :3]

    return position, rotation

# cconverting rotation matrix to eular''s angels(orientation)
from scipy.spatial.transform import Rotation as R
def rotation_matrix_to_euler(R_mat):
    r = R.from_matrix(R_mat)
    euler_angles = r.as_euler('xyz', degrees=True)  # or 'zyx' depending on convention
    return euler_angles  # returns rx, ry, rz


# Example usage
dataset=[]
for i in range (0,len(θ1)):
  angles=[θ1[i],θ2[i],θ3[i],θ4[i],θ5[i],θ6[i]]
  pos, rot = fk_irb4600(angles)
  euler_angles = rotation_matrix_to_euler(rot)
  res = np.concatenate((pos, euler_angles,angles))
  # print(res)
  dataset.append(res)

print(dataset[0])


[1351.75687652  -67.79566264  784.00878248  -16.56434327   12.88821111
 -125.16985813    0.           40.           51.42304845  200.
   82.35281374  235.11410092]


In [ ]:
# perplexity df parameterrs

def dh_matrix(theta, d, a, alpha):
    ct, st = np.cos(theta), np.sin(theta)
    ca, sa = np.cos(alpha), np.sin(alpha)
    return np.array([
        [ct, -st*ca, st*sa, a*ct],
        [st, ct*ca, -ct*sa, a*st],
        [0, sa, ca, d],
        [0, 0, 0, 1]
    ])

# forward kinemarics
def fk_irb4600(joint_angles_deg):
    theta = np.radians(joint_angles_deg)

    # DH parameters
    dh_params = [
        [theta[0], 495, 0, np.pi/2],
        [theta[1], 0, 1250, 0],
        [theta[2], 0, 350, 0],
        [theta[3], 1250, 0, np.pi/2],
        [theta[4], 0, 0, -np.pi/2],
        [theta[5], 80, 0, 0]
    ]

    T = np.eye(4)
    for p in dh_params:
        T = T @ dh_matrix(*p)
    # print(T)
    position = T[:3, 3]
    rotation = T[:3, :3]

    return position, rotation

# cconverting rotation matrix to eular''s angels(orientation)
from scipy.spatial.transform import Rotation as R
def rotation_matrix_to_euler(R_mat):
    r = R.from_matrix(R_mat)
    euler_angles = r.as_euler('xyz', degrees=True)  # or 'zyx' depending on convention
    return euler_angles  # returns rx, ry, rz


# Example usage
dataset=[]
for i in range (0,len(θ1)):
  angles=[θ1[i],θ2[i],θ3[i],θ4[i],θ5[i],θ6[i]]
  pos, rot = fk_irb4600(angles)
  euler_angles = rotation_matrix_to_euler(rot)
  res = np.concatenate((pos, euler_angles,angles))
  # print(res)
  dataset.append(res)

print(dataset)


In [8]:
import pandas as pd

columns = ['x', 'y', 'z', 'roll(rx)', 'pitch(ry)', 'yaw(rz)',
           'j1', 'j2', 'j3', 'j4', 'j5', 'j6']

df = pd.DataFrame(dataset, columns=columns)
df.to_csv('IK_sinusoidal_dataset_irb1520D.csv', index=False)